In [1]:
import tensorflow as tf
import tensorflow_text as text
import functools
print("TensorFlow version: ", tf.__version__)


2023-09-21 11:16:08.214985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 11:16:10.765646: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


TensorFlow version:  2.11.0


In [2]:
examples = {
    "text_a": [
      "Sponge bob Squarepants is an Avenger",
      "Marvel Avengers"
    ],
    "text_b": [
     "Barack Obama is the President.",
     "President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))


2023-09-21 11:16:27.052988: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 11:16:27.350410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 11:16:27.350923: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-21 11:16:27.396637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

In [6]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5

_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
          value_dtype=tf.int64
        ),
      num_oov_buckets=1
)


In [7]:
random_selector = text.RandomItemSelector(
    max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH,
    selection_rate=0.2,
    unselectable_ids=[_START_TOKEN, _END_TOKEN, _UNK_TOKEN]
)

In [8]:
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, 0.8)


In [9]:
def bert_pretrain_preprocess(vocab_table, features):
    # Input is a string Tensor of documents, shape [batch, 1].
    text_a = features["text_a"]
    text_b = features["text_b"]

    # Tokenize segments to shape [num_sentences, (num_words)] each.
    tokenizer = text.BertTokenizer(
      vocab_table,
      token_out_type=tf.int64)
    segments = [tokenizer.tokenize(text).merge_dims(
      1, -1) for text in (text_a, text_b)]

    # Truncate inputs to a maximum length.
    trimmer = text.RoundRobinTrimmer(max_seq_length=6)
    trimmed_segments = trimmer.trim(segments)

    # Combine segments, get segment ids and add special tokens.
    segments_combined, segment_ids = text.combine_segments(
      trimmed_segments,
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)

    # Apply dynamic masking task.
    masked_input_ids, masked_lm_positions, masked_lm_ids = (
      text.mask_language_model(
        segments_combined,
        random_selector,
        mask_values_chooser,
      )
    )

    # Prepare and pad combined segment inputs
    input_word_ids, input_mask = text.pad_model_inputs(
    masked_input_ids, max_seq_length=_MAX_SEQ_LEN)
    input_type_ids, _ = text.pad_model_inputs(
    segment_ids, max_seq_length=_MAX_SEQ_LEN)

    # Prepare and pad masking task inputs
    masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
    masked_lm_positions, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
    masked_lm_ids, _ = text.pad_model_inputs(
    masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

    model_inputs = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids,
      "masked_lm_ids": masked_lm_ids,
      "masked_lm_positions": masked_lm_positions,
      "masked_lm_weights": masked_lm_weights,
    }
    return model_inputs

In [10]:
dataset = (
    tf.data.Dataset.from_tensors(examples)
    .map(functools.partial(bert_pretrain_preprocess, lookup_table))
)

next(iter(dataset))


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  1, 24,  4, 27,  5, 19],
        [ 3, 18,  1, 15,  4, 21, 28, 17]])>,
 'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[ 9, 16,  0,  0,  0],
        [12, 30,  0,  0,  0]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[2, 5, 0, 0, 0],
        [2, 7, 0, 0, 0]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]])>}

In [49]:
class BertEmbedding(tf.keras.layers.Layer):
    """Bert embedding is composed of a positional embedding layer, a segment embedding layer and a normal embedding layer

    Input shape
      - token index 2D tensor with shape: ``(batch_size, sequence_length)``.
      - segment index 2D tensor with shape: ``(batch_size, sequence_length)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(self, vocab_size, seq_length=128, dim=512, **kwargs):
        super(BertEmbedding, self).__init__(**kwargs)
        self.length = seq_length
        self.dim = dim
        self.token_emb = tf.keras.layers.Embedding(
            input_dim=vocab_size, output_dim=dim, mask_zero=True
        )
        self.position_emb = tf.keras.layers.Embedding(input_dim=seq_length, output_dim=dim)

    def call(self, inputs, **kwargs):
        tokens = inputs["input_word_ids"]
        length = tf.shape(tokens)[1]
        embedded_tokens = self.token_emb(tokens)
        embedded_positions = self.position_emb(tf.range(length))
        # This factor sets the relative scale of the embedding and positonal_encoding.
        embedded_tokens *= tf.math.sqrt(tf.cast(self.dim, tf.float32))
        return (
            embedded_tokens + embedded_positions[tf.newaxis, :, :]
        )

    # Pass mask from token_emb, https://www.tensorflow.org/guide/keras/understanding_masking_and_padding#supporting_masking_in_your_custom_layers
    def compute_mask(self, inputs, mask=None):
        return inputs["input_mask"]


In [50]:
bert = BertEmbedding(len(_VOCAB), _MAX_SEQ_LEN)

In [13]:
bert(next(iter(dataset)))

<tf.Tensor: shape=(2, 8, 512), dtype=float32, numpy=
array([[[ 0.28828734, -0.9332832 , -0.7811977 , ..., -0.6922506 ,
          0.77401567,  0.93710345],
        [ 0.2757414 , -0.69747627, -0.72298133, ..., -1.0327996 ,
          0.283246  , -0.0515362 ],
        [-0.69469255, -0.20174801,  0.39593172, ...,  0.3419567 ,
          1.0862778 , -1.0407891 ],
        ...,
        [-0.8904924 ,  1.0859271 , -0.75398684, ..., -0.8904877 ,
          0.41478336,  1.0157971 ],
        [-0.33193454,  1.1266761 , -0.00971618, ...,  0.07721391,
          0.38347268,  1.0570387 ],
        [ 0.44425493,  0.85371727, -0.57546055, ..., -0.29008284,
         -0.7427849 ,  0.32641366]],

       [[ 0.28828734, -0.9332832 , -0.7811977 , ..., -0.6922506 ,
          0.77401567,  0.93710345],
        [-0.9132184 , -1.112237  ,  0.47554365, ..., -0.04587528,
          0.47510335,  0.8591468 ],
        [-0.69469255, -0.20174801,  0.39593172, ...,  0.3419567 ,
          1.0862778 , -1.0407891 ],
        ...,
 

In [14]:
%load_ext autoreload
%autoreload 2
from trainer.models.common.transformer import Encoder

In [51]:
class Bert(tf.keras.layers.Layer):
    """Bert model is a stack of multiple layers of encoders

    Input shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    References
        - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
    """

    def __init__(
        self,
        vocab_size,
        seq_length,
        layer_num=6,
        model_dim=512,
        ff_dim=2048,
        dropout=0.1,
        head_num=8,
        **kwargs,
    ):
        super(Bert, self).__init__(**kwargs)
        self.emb = BertEmbedding(
            vocab_size=vocab_size, seq_length=seq_length, dim=model_dim
        )
        self.encoders = [
            Encoder(
                model_dim=model_dim, ff_dim=ff_dim, dropout=dropout, head_num=head_num
            )
            for _ in range(layer_num)
        ]

    def call(self, inputs, training=False):
        # shape [batch_size, token_length]
        emb = self.emb(inputs, training=training)
        # shape [batch_size, token_length, model_dim]
        for encoder in self.encoders:
            emb = encoder(emb, training=training)
        # shape [batch_size, token_length, model_dim]
        return emb


class Bert4Rec(tf.keras.layers.Layer):
    """Masked language model simply mask some percentage of the input tokens at random, and then predict those masked tokens

    Input shape
      - 3D tensor with shape: ``(batch_size, sequence_length, embedding_size)``.

    Output shape
      - 3D tensor with shape: ``(batch_size, masked_positions, vocab_size)``.

    References
        - [BERT4Rec: Sequential Recommendation with Bidirectional Encoder Representations from Transformer](https://arxiv.org/pdf/1904.06690.pdf)
    """

    def __init__(
        self,
        vocab_size,
        seq_length,
        layer_num=6,
        model_dim=512,
        ff_dim=2048,
        dropout=0.1,
        head_num=8,
        **kwargs,
    ):
        super(Bert4Rec, self).__init__(**kwargs)

        self.bert = Bert(
            vocab_size=vocab_size,
            seq_length=seq_length,
            layer_num=layer_num,
            model_dim=model_dim,
            ff_dim=ff_dim,
            dropout=dropout,
            head_num=head_num,
            **kwargs,
        )
        # shape [vocab_size, embedding_size]
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.dense = tf.keras.layers.Dense(
            model_dim, activation='gelu'
        )

    def call(self, inputs, training=False):
        # shape [batch_size, token_length, model_dim]
        emb = self.dropout(self.bert(inputs, training=training))
        # shape [batch_size, token_length, model_dim]
        emb = self.dense(emb)
        # gather the corresponding logits per the masked_lm_positions
        # shape [batch_size, masked_lm_positions, model_dim]
        emb = tf.gather(emb, inputs["masked_lm_positions"], axis=1, batch_dims=1)
        # shape [batch_size, vocab_size, embedding_size]
        input_emb_weights = self.bert.emb.token_emb.trainable_variables[0]
        # shape [batch_size, masked_positions, vocab_size]
        return tf.matmul(emb, input_emb_weights, transpose_b=True)



In [52]:
mlm = Bert4Rec(len(_VOCAB), _MAX_SEQ_LEN)
mlm(next(iter(dataset)))

<tf.Tensor: shape=(2, 5, 31), dtype=float32, numpy=
array([[[-1.204608  , -0.3435057 , -0.02068051, -0.23195934,
         -0.04339382, -0.30372506,  0.3473095 , -0.05549674,
          0.1678929 ,  0.18790339, -0.7526599 ,  0.10378174,
         -0.22640207,  0.52210367,  0.12653294,  0.15099859,
         -0.27592474, -0.21125063,  0.12434558,  0.2426585 ,
          1.3169292 ,  0.5647949 ,  0.1033027 ,  0.63603616,
         -0.5432624 ,  0.13483912,  0.20665452, -0.26990935,
          0.01863711,  0.40742958, -0.14430434],
        [-1.2091808 , -0.33642054, -0.0244432 , -0.22224677,
         -0.0459226 , -0.30168685,  0.33974928, -0.07155635,
          0.15827467,  0.19008195, -0.76364845,  0.09578253,
         -0.22988907,  0.5317125 ,  0.12016733,  0.1648632 ,
         -0.27785444, -0.20776488,  0.10678308,  0.24328533,
          1.3316113 ,  0.5694369 ,  0.10107972,  0.6305535 ,
         -0.53755385,  0.14121746,  0.2001932 , -0.2647997 ,
          0.02041976,  0.39034614, -0.1451791

In [53]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False, reduction="none"
    )
    loss = loss_object(label, pred)
    
    # mask indices where label == 0 (padding)
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    # get the prediction index for target token
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match) / tf.reduce_sum(mask)


In [54]:
class BertModel(tf.keras.Model):

    def call(self, inputs, training=False) -> tf.Tensor:
        logits = mlm(inputs, training=training)
        try:
            # Drop the keras mask, so it doesn't scale the losses/metrics.
            # b/250038731
            del logits._keras_mask
        except AttributeError:
            pass
        return logits


In [55]:
bert_model = BertModel()

In [56]:
bert_model.compile(
            loss=masked_loss,
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=0.0001,
            ),
            metrics=[masked_accuracy],
        )

In [57]:
bert_model.fit(dataset.repeat(1000).map(lambda x: (x, x["masked_lm_ids"])))

1000/1000 [==============================] - 16s 13ms/step - loss: 10.9292 - masked_accuracy: 7.5000e-04
